In [ ]:
import sys
import os
import pandas as pd
import re

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

In [ ]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [ ]:
#test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")


In [ ]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [ ]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [ ]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']

In [ ]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [ ]:
test_train_df

In [ ]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [ ]:
from dev_wtp_io_utils import import_sst_cond_w1_subjs_to_pkl

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs[0:2],'conditions',
    out_folder = ml_data_folderpath + "/")

OK, not bad, so we have learned to load data, and show it. We might be best to re-write the code, though, to use the first image as the image series's own mask--using `nilearn.masking.compute_brain_mask`, rather than worry about why the input mask isn't quite aligned. 

In [ ]:
len(train_subjs)

### With supplementary data

In [ ]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1
#individual_differences['wave'] = individual_differences['wave'].astype(object) # for compatibility with the wave column in the dataset

In [ ]:
from outlier_detection import *

In [ ]:
"', '".join(individual_differences.columns)

In [ ]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_spatially_concatenated',
    concatenate_condition_labels=True
)

## concatenating conditions

In [ ]:
from nilearn import plotting
from nilearn.masking import compute_brain_mask
import nltools as nlt
import nilearn as nil

In [ ]:
from nltools import Brain_Data

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_spatially_concatenated',
    concatenate_condition_labels=True
)

In [ ]:
out_data = pickle.load(open(
    '/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml/SST/Brain_Data_conditions_2subs_correct_cond_w_supplementary_mask_fix.pkl','rb'
))

In [ ]:
out_data.shape

In [ ]:
plotting.plot_glass_brain(out_data[0,].to_nifti())

## post-error conditions

In [ ]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq

train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond_w_supplementary'
)

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected[0:2],'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_go_fcs_w_supplementary'
)

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_correct_stop_cg_w_supplementary'
)

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingFailedStop'],
    condition_count_required=1,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_failed_stop_cg_failed_stop_w_supplementary'
)

In [ ]:
import_sst_cond_w1_subjs_to_pkl(
    train_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_stop_cg_spatially_concatenated',
    concatenate_condition_labels=True
)